## Building the model

In [ ]:
import utils

In [2]:
from utils import train_csv, test_csv

from utils import train_capped, train_nocap
from utils import train_log_transformed, train_log_transformed_capped, train_log_transformed_nocap
from utils import train_absmax_transformed, train_absmax_transformed_capped, train_absmax_transformed_nocap
from utils import train_zscore_transformed, train_zscore_transformed_capped, train_zscore_transformed_nocap
from utils import train_minmax_transformed, train_minmax_transformed_capped, train_minmax_transformed_nocap
from utils import train_yj_transformed, train_yj_transformed_capped, train_yj_transformed_nocap

from utils import test_csv, test_minmax_transformed, test_absmax_transformed, test_log_transformed, test_zscore_transformed, test_yj_transformed

from utils import test_model_rmse, check_residuals, data_transform, iterative_impute

In [5]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import ElasticNet

import numpy as np
import os
import seaborn as sns
import pandas as pd

In [18]:
alphas = np.concatenate((np.logspace(-4,0,20), np.arange(1, 100, 1).tolist()))
max_iter = list(range(1000,10001,1000))
l1_ratio = np.arange(0.0, 1.0, 0.05).tolist()
tol = [0.0001, 0.01, 0.1, 0.5]

X = train_csv.iloc[:,:-1]
Y = train_csv.iloc[:,-1]

elasticnet_gscv = GridSearchCV(estimator=ElasticNet(), 
                                param_grid={'alpha': alphas,
                                            'max_iter': max_iter,
                                            'l1_ratio': l1_ratio,
                                            'tol':tol},   
                                scoring='neg_root_mean_squared_error',
                                cv=5)

In [19]:
elasticnet_gscv.fit(X, Y)
elasticnet_gscv.best_params_

C:\Users\yixin\AppData\Roaming\Python\Python310\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.998e+11, tolerance: 7.592e+08 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
C:\Users\yixin\AppData\Roaming\Python\Python310\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.200e+11, tolerance: 7.288e+08 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of th

KeyboardInterrupt: 

In [41]:
from time import perf_counter

In [60]:
def test_elasticnet_gscv(df, df_name = None, result_to_txt = False):

    X = df.iloc[:,:-1]
    Y = df.iloc[:,-1]
    
    alphas = np.concatenate((np.logspace(-4,0,10), np.arange(1, 100, 10).tolist()))
    max_iter = list(range(1000,10001,1000))
    l1_ratio = np.arange(0.0, 1.0, 0.05).tolist()
    tol = [0.0001, 0.01, 0.1, 0.5]

    param_grid={'alpha': alphas,
            'max_iter': max_iter,
            'l1_ratio': l1_ratio,
            'tol':tol,
            'random_state': [50]}
    
    elasticnet_gscv = GridSearchCV(estimator=ElasticNet(), param_grid=param_grid, scoring='neg_root_mean_squared_error', cv=5, n_jobs=-1)

    elasticnet_gscv.fit(X,Y)
    
    print(f"Best parameter grids: {elasticnet_gscv.best_params_}")
    print(f"Score: {elasticnet_gscv.best_score_}")

    if result_to_txt:
        best_record_sentence = f"{df_name}: {elasticnet_gscv.best_score_}, {elasticnet_gscv.best_params_} \n"
    
        with open("result.txt", "a") as outfile:
            outfile.write(best_record_sentence)

In [44]:
start = perf_counter()
test_elasticnet_gscv(train_csv, 'raw_training_Data', result_to_txt=True)
end = perf_counter()

print(f"Seconds to evaluate this model: {end-start}")

Best parameter grids: {'alpha': 0.0001, 'l1_ratio': 0.0, 'max_iter': 9000, 'random_state': 50, 'tol': 0.0001}
Score: -35489.58460891884
Seconds to evaluate this model: 153.76287219999358


C:\Users\yixin\AppData\Roaming\Python\Python310\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.023e+11, tolerance: 9.208e+08 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(


## Evaluation on various datasets

In [62]:
### Preparing the datasets

### Evaluate on the ones without outliers first, then the purely transformed ones


models_map = dict()
 
for lower_bond in [1,2]:
    for transformation_mode in ["min-max", "abs-max", "z-score", "log",  "yeo-johnson",  "sqrt"]:
    
        original_train = train_csv.copy()
        
        data_name = f"train_{transformation_mode}_transformed_97_{lower_bond}_cap"
        data = utils.data_transform(original_train, transformation_mode)
        data = utils.remove_outliers_capping(data, lower_bond=lower_bond/100, upper_bond=0.97)
        data = utils.iterative_impute(data)
        models_map.setdefault(data_name, data)

for upper_bond in [98,99]: 
    for lower_bond in [0,1,2]:
        for transformation_mode in ["min-max", "abs-max", "z-score", "log",  "yeo-johnson",  "sqrt"]:
        
            original_train = train_csv.copy()
            
            data_name = f"train_{transformation_mode}_transformed_{upper_bond}_{lower_bond}_cap"
            data = utils.data_transform(original_train, transformation_mode)
            data = utils.remove_outliers_capping(data, lower_bond=lower_bond/100, upper_bond=upper_bond/100)
            data = utils.iterative_impute(data)
            models_map.setdefault(data_name, data)

[IterativeImputer] Completing matrix with shape (939, 80)
[IterativeImputer] Change: 0.0, scaled tolerance: 426.0 
[IterativeImputer] Early stopping criterion reached.
[IterativeImputer] Completing matrix with shape (939, 80)
[IterativeImputer] Change: 0.0, scaled tolerance: 426.0 
[IterativeImputer] Early stopping criterion reached.
[IterativeImputer] Completing matrix with shape (939, 80)
[IterativeImputer] Change: 0.0, scaled tolerance: 426.0 
[IterativeImputer] Early stopping criterion reached.
[IterativeImputer] Completing matrix with shape (941, 80)


C:\Users\yixin\AppData\Roaming\Python\Python310\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


[IterativeImputer] Change: 1.1890853268024786, scaled tolerance: 426.0 
[IterativeImputer] Early stopping criterion reached.


C:\Users\yixin\AppData\Roaming\Python\Python310\site-packages\numpy\core\_methods.py:176: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)
C:\Users\yixin\AppData\Roaming\Python\Python310\site-packages\numpy\core\_methods.py:187: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(x, axis, dtype, out, keepdims=keepdims, where=where)


[IterativeImputer] Completing matrix with shape (939, 80)
[IterativeImputer] Change: 0.0, scaled tolerance: 5.0798501647650995e+150 
[IterativeImputer] Early stopping criterion reached.
[IterativeImputer] Completing matrix with shape (941, 80)


...

[IterativeImputer] Change: 1.2083726030194728, scaled tolerance: 556.581 
[IterativeImputer] Early stopping criterion reached.


C:\Users\yixin\AppData\Roaming\Python\Python310\site-packages\numpy\core\_methods.py:176: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)
C:\Users\yixin\AppData\Roaming\Python\Python310\site-packages\numpy\core\_methods.py:187: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(x, axis, dtype, out, keepdims=keepdims, where=where)


[IterativeImputer] Completing matrix with shape (1193, 80)
[IterativeImputer] Change: 0.0, scaled tolerance: 5.0798501647650995e+150 
[IterativeImputer] Early stopping criterion reached.
[IterativeImputer] Completing matrix with shape (1173, 80)


C:\Users\yixin\AppData\Roaming\Python\Python310\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)


[IterativeImputer] Change: 3.7698739177808402, scaled tolerance: 556.581 
[IterativeImputer] Early stopping criterion reached.


In [64]:
for df_name, df in models_map.items():
    test_elasticnet_gscv(df, df_name=df_name, result_to_txt=True)

Best parameter grids: {'alpha': 0.002154434690031882, 'l1_ratio': 0.8500000000000001, 'max_iter': 1000, 'random_state': 50, 'tol': 0.0001}
Score: -20132.229968880518
Best parameter grids: {'alpha': 0.000774263682681127, 'l1_ratio': 0.65, 'max_iter': 1000, 'random_state': 50, 'tol': 0.01}
Score: -20106.692295520428
Best parameter grids: {'alpha': 0.12915496650148828, 'l1_ratio': 0.5, 'max_iter': 1000, 'random_state': 50, 'tol': 0.0001}
Score: -19989.013488019147
Best parameter grids: {'alpha': 0.005994842503189409, 'l1_ratio': 0.25, 'max_iter': 1000, 'random_state': 50, 'tol': 0.01}
Score: -21021.750966198317


C:\Users\yixin\AppData\Roaming\Python\Python310\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.753e+11, tolerance: 2.802e+08
  model = cd_fast.enet_coordinate_descent(


Best parameter grids: {'alpha': 0.002154434690031882, 'l1_ratio': 0.1, 'max_iter': 1000, 'random_state': 50, 'tol': 0.0001}
Score: -21052.447254822655


C:\Users\yixin\AppData\Roaming\Python\Python310\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.615e+11, tolerance: 2.803e+08
  model = cd_fast.enet_coordinate_descent(


Best parameter grids: {'alpha': 0.016681005372000592, 'l1_ratio': 0.35000000000000003, 'max_iter': 1000, 'random_state': 50, 'tol': 0.0001}
Score: -20259.263322929342
Best parameter grids: {'alpha': 0.000774263682681127, 'l1_ratio': 0.45, 'max_iter': 1000, 'random_state': 50, 'tol': 0.01}
Score: -20555.082831266343
Best parameter grids: {'alpha': 0.000774263682681127, 'l1_ratio': 0.55, 'max_iter': 1000, 'random_state': 50, 'tol': 0.01}
Score: -20527.576924293146
Best parameter grids: {'alpha': 0.12915496650148828, 'l1_ratio': 0.35000000000000003, 'max_iter': 1000, 'random_state': 50, 'tol': 0.0001}
Score: -20383.18151683696
Best parameter grids: {'alpha': 0.046415888336127774, 'l1_ratio': 0.9, 'max_iter': 1000, 'random_state': 50, 'tol': 0.01}
Score: -21357.682456785024


...

C:\Users\yixin\AppData\Roaming\Python\Python310\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.914e+11, tolerance: 5.136e+08
  model = cd_fast.enet_coordinate_descent(


Best parameter grids: {'alpha': 0.016681005372000592, 'l1_ratio': 0.35000000000000003, 'max_iter': 1000, 'random_state': 50, 'tol': 0.0001}
Score: -24158.680122392325
Best parameter grids: {'alpha': 0.000774263682681127, 'l1_ratio': 0.35000000000000003, 'max_iter': 1000, 'random_state': 50, 'tol': 0.0001}
Score: -23872.119329343703
Best parameter grids: {'alpha': 0.002154434690031882, 'l1_ratio': 0.8, 'max_iter': 1000, 'random_state': 50, 'tol': 0.01}
Score: -23868.486811530718
Best parameter grids: {'alpha': 0.12915496650148828, 'l1_ratio': 0.25, 'max_iter': 1000, 'random_state': 50, 'tol': 0.01}
Score: -23709.902075423724
Best parameter grids: {'alpha': 0.016681005372000592, 'l1_ratio': 0.65, 'max_iter': 1000, 'random_state': 50, 'tol': 0.01}
Score: -25440.13358321226


C:\Users\yixin\AppData\Roaming\Python\Python310\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.259e+11, tolerance: 5.038e+08
  model = cd_fast.enet_coordinate_descent(


Best parameter grids: {'alpha': 0.002154434690031882, 'l1_ratio': 0.2, 'max_iter': 7000, 'random_state': 50, 'tol': 0.0001}
Score: -24843.70277731159
Best parameter grids: {'alpha': 0.016681005372000592, 'l1_ratio': 0.25, 'max_iter': 1000, 'random_state': 50, 'tol': 0.0001}
Score: -24556.727309581867


C:\Users\yixin\AppData\Roaming\Python\Python310\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.891e+11, tolerance: 4.935e+08
  model = cd_fast.enet_coordinate_descent(
